In [225]:
import pandas as pd
import numpy as np
import zipfile

In [226]:
with zipfile.ZipFile("../data/reddit_scrape.csv.zip") as z:
    with z.open("reddit_scrape.csv") as f:
        reddit_df = pd.read_csv(f)

reddit_df['Date'] = pd.to_datetime(reddit_df['Date'])    

reddit_df.head()

,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,...,Notes,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,GameHub Scraped,Game thread scraped,Post game thread Scraped,Subscribers (in thousands)
0,2017,1,Thu,2017-09-07,8:30,@,New England Patriots,L 27-42,L -8,NaN,...,NaN,NaN,NaN,NaN,https://www.reddit.com/r/nfl/comments/6yr619/g...,https://www.reddit.com/r/nfl/comments/6ysc10/p...,NaN,"[3193, ('Dear Falcons fans: are you guys ready...","[2781, ('I\'m imagining a world where the Patr...",640
1,2017,1,Sun,2017-09-10,1:00,@,Buffalo Bills,W 21-12,W -7,NaN,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97il/g...,https://www.reddit.com/r/nfl/comments/6zajs1/p...,"[240, ('Bitter-sweet start to the NFL season w...","[109, ('I\'m a Jets fan. I\'m a Texas A&M fan....","[120, ('Bills number 1 seed in the AFCE!', 194...",640
2,2017,1,Sun,2017-09-10,1:00,NaN,Atlanta Falcons,W 23-17,L -6.5,@,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97gn/g...,https://www.reddit.com/r/nfl/comments/6zajuc/p...,"[240, ('Bitter-sweet start to the NFL season w...","[321, ('Wow this national anthem singer is a f...","[351, (""We just went down to the wire against ...",640
3,2017,1,Sun,2017-09-10,1:00,@,Cincinnati Bengals,L 0-20,L -2.5,NaN,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97hl/g...,https://www.reddit.com/r/nfl/comments/6zajr0/p...,"[240, ('Bitter-sweet start to the NFL season w...","[192, (""Man I hope Woodhead stays healthy for ...","[293, ('[Summary of this game](https://pbs.twi...",640
4,2017,1,Sun,2017-09-10,1:00,NaN,Pittsburgh Steelers,W 21-18,L -10,@,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97l9/g...,https://www.reddit.com/r/nfl/comments/6zajwi/p...,"[240, ('Bitter-sweet start to the NFL season w...","[367, (""1 drive no turnovers. WE'VE TURNED THE...","[383, (""Don't let this distract you from the f...",640


In [227]:
scrape_cols = [x for x in reddit_df.columns if 'craped' in x]
for col in scrape_cols: # converts strings to lists
    reddit_df[col] = reddit_df[col].apply(lambda x: eval(x) if type(x)==str else x)

In [228]:
text = reddit_df['Game thread scraped'][0][1][0]
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. \n\nSincerely, \n\n\nSeahawks fans everywhere.'

In [229]:
from markdown import Markdown
from io import StringIO


def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()


# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text).replace(" \n"," ")

text = unmark(text).replace(" \n"," ")
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. Sincerely, Seahawks fans everywhere.'

In [230]:
import string
text = text.translate(str.maketrans('', '', string.punctuation))
text

'Dear Falcons fans are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl I feel your pain Sincerely Seahawks fans everywhere'

In [231]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 202.1 kB/s eta 0:00:0000:0100:01


In [232]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [233]:
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
    elif sentiment_dict['compound'] <= - 0.05 :
        print("Negative")
    else :
        print("Neutral")

In [234]:
sentiment_scores(text)

Overall sentiment dictionary is :  {'neg': 0.18, 'neu': 0.492, 'pos': 0.328, 'compound': 0.6486}
sentence was rated as  18.0 % Negative
sentence was rated as  49.2 % Neutral
sentence was rated as  32.800000000000004 % Positive
Sentence Overall Rated As Positive


In [235]:
text = reddit_df['Game thread scraped'][0][2][0]
print(text)
text = unmark(text).replace(" \n"," ")
text = text.translate(str.maketrans('', '', string.punctuation))
sentiment_scores(text)

#Fuckin Mark Wahlberg LEFT THE SUPER BOWL EARLY BECAUSE THE PATS WERE DOWN 28-3
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.6841}
sentence was rated as  0.0 % Negative
sentence was rated as  73.7 % Neutral
sentence was rated as  26.3 % Positive
Sentence Overall Rated As Positive


In [236]:
import re

def convert_super_bowl(text):
    # Use regular expression to find and replace "super bowl" with "superbowl" (case-insensitive)
    modified_text = re.sub(r'\b(super)\s+(bowl)\b', r'\1\2', text, flags=re.IGNORECASE)
    return modified_text

# Example usage:
original_text = "The Super Bowl is a great event."
converted_text = convert_super_bowl(original_text)
print(converted_text)


The SuperBowl is a great event.


In [289]:
import random

thread_num = []
comment_num = []
comments = []
pos_words = []
neu_words = []
neg_words = []
scores = []
overall_rating = []

for i in range(10):
    num_1 = random.randrange(len(reddit_df))
    for j in range(50):
        thread_num.append(num_1)
        num_2 = random.randrange(len(reddit_df.iloc[num_1,:].loc['Game thread scraped']))
        if j != 0:
            while num_2 in comment_num[-j:]:
                num_2 = random.randrange(len(reddit_df.iloc[num_1,:].loc['Game thread scraped']))
        comment_num.append(num_2)
        text = reddit_df['Game thread scraped'][num_1][num_2][0]
        # print(text)
        # print("____________________________________________________________________________________________________")
        comments.append(text)
        text = unmark(text).replace(" \n"," ")
        text = convert_super_bowl(text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        from nltk.tokenize import word_tokenize, RegexpTokenizer
        from nltk.sentiment.vader import SentimentIntensityAnalyzer
        tokenized_sentence = nltk.word_tokenize(text)

        sid = SentimentIntensityAnalyzer()
        pos_word_list=[]
        neu_word_list=[]
        neg_word_list=[]

        for word in tokenized_sentence:
            if (sid.polarity_scores(word)['compound']) >= 0.05:
                pos_word_list.append(word)
            elif (sid.polarity_scores(word)['compound']) <= -0.05:
                neg_word_list.append(word)
            else:
                neu_word_list.append(word)                

        # print('Positive:', pos_word_list) 
        pos_words.append(pos_word_list)
        # print('Neutral:', neu_word_list) 
        neu_words.append(neu_word_list)
        # print('Negative:', neg_word_list) 
        neg_words.append(neg_word_list)
        score = sid.polarity_scores(text)
        # print('\nScores:', score,"\n")
        scores.append(score)
        # print("Sentence Overall Rated As", end = " ")
        if score['compound'] >= 0.05 :
            # print("Positive")
            overall_rating.append(1)
        elif score['compound'] <= - 0.05 :
            # print("Negative")
            overall_rating.append(-1)
        else :
            # print("Neutral")
            overall_rating.append(0)

In [292]:
data = {
    "Thread": thread_num,
    "Comment Number": comment_num,
    'Comment Text': comments,
    'Positive Words': pos_words,
    'Neutral Words': neu_words,
    'Negative Words': neg_words,
    'Scores': scores,
    'Overall Rating': overall_rating
}

verifying_df = pd.DataFrame(data)
verifying_df#.to_csv('verifying_scores.csv')

,Thread,Comment Number,Comment Text,Positive Words,Neutral Words,Negative Words,Scores,Overall Rating
0,220,251,He broke crossed the line and had control. How...,[],"[He, crossed, the, line, and, had, control, Ho...","[broke, lose]","{'neg': 0.103, 'neu': 0.813, 'pos': 0.083, 'co...",-1
1,220,396,There were multiple Patriots not lined up prop...,"[play, like]","[There, were, multiple, Patriots, not, lined, ...",[miss],"{'neg': 0.056, 'neu': 0.772, 'pos': 0.172, 'co...",1
2,220,52,THIS WEEK ON WHAT IS A CATCH,[],"[THIS, WEEK, ON, WHAT, IS, A, CATCH]",[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0
3,220,420,8 is neither 5 or 6. Good job D.,[Good],"[8, is, neither, 5, or, 6, job, D]",[],"{'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'comp...",-1
4,220,283,That's a catch. Don't care what the rules say.,[care],"[Thats, a, catch, Dont, what, the, rules, say]",[],"{'neg': 0.273, 'neu': 0.727, 'pos': 0.0, 'comp...",-1
...,...,...,...,...,...,...,...,...
495,55,189,Wanna know what's gonna happen? Carolina is go...,[win],"[Wan, na, know, whats, gon, na, happen, Caroli...","[fuck, wrong, forgotten]","{'neg': 0.134, 'neu': 0.806, 'pos': 0.06, 'com...",-1
496,55,403,I'm loving these TD celebrations. Glad they're...,"[loving, Glad]","[Im, these, TD, celebrations, theyre, back]",[],"{'neg': 0.0, 'neu': 0.465, 'pos': 0.535, 'comp...",1
497,55,50,Love this hate Patriot fans.,[Love],"[this, Patriot, fans]",[hate],"{'neg': 0.339, 'neu': 0.275, 'pos': 0.385, 'co...",1
498,55,215,STEPHEN GOSTKOWSKI IS STILL PERFECT THIS MONTH!,[PERFECT],"[STEPHEN, GOSTKOWSKI, IS, STILL, THIS, MONTH]",[],"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp...",1


In [344]:
df = pd.read_csv('verifying_scores.csv').iloc[:,1:]
def cutoffs(lower_bound, upper_bound, col):
    lst = []
    for i in col:
        if i <= lower_bound:
            lst.append(-1)
        elif i >= upper_bound:
            lst.append(1)
        else:
            lst.append(0)
    return lst
df['Scores'] = df['Scores'].apply(eval)
df['Compound'] = [i['compound'] for i in df['Scores']]
#df['Overall Rating'] = cutoffs(-0.05, 0.05,df['Compound'])
df['Accurate'] = (df['My rating'] == df['Overall Rating']).astype(int)
df.loc[df['Overall Rating'] == 0, 'Accurate'] = df.loc[df['Overall Rating'] == 0, 'Accurate']/2
df.loc[df['My rating'] == -99, 'Accurate'] = -99
df.loc[df['My rating'].isna(), 'Accurate'] = np.NaN
print(df['Accurate'].value_counts())
df

 1.0     33
 0.0     21
-99.0    12
 0.5      3
Name: Accurate, dtype: int64


,Thread,Comment Number,Comment Text,Positive Words,Neutral Words,Negative Words,Scores,Overall Rating,My rating,Compound,Accurate
0,220,251,He broke crossed the line and had control. How...,[],"['He', 'crossed', 'the', 'line', 'and', 'had',...","['broke', 'lose']","{'neg': 0.103, 'neu': 0.813, 'pos': 0.083, 'co...",-1,-1.0,-0.1386,1.0
1,220,396,There were multiple Patriots not lined up prop...,"['play', 'like']","['There', 'were', 'multiple', 'Patriots', 'not...",['miss'],"{'neg': 0.056, 'neu': 0.772, 'pos': 0.172, 'co...",1,-1.0,0.5106,0.0
2,220,52,THIS WEEK ON WHAT IS A CATCH,[],"['THIS', 'WEEK', 'ON', 'WHAT', 'IS', 'A', 'CAT...",[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,0.0,0.0000,0.5
3,220,420,8 is neither 5 or 6. Good job D.,['Good'],"['8', 'is', 'neither', '5', 'or', '6', 'job', ...",[],"{'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'comp...",-1,-99.0,-0.3412,-99.0
4,220,283,That's a catch. Don't care what the rules say.,['care'],"['Thats', 'a', 'catch', 'Dont', 'what', 'the',...",[],"{'neg': 0.273, 'neu': 0.727, 'pos': 0.0, 'comp...",-1,1.0,-0.3875,0.0
...,...,...,...,...,...,...,...,...,...,...,...
491,55,189,Wanna know what's gonna happen? Carolina is go...,['win'],"['Wan', 'na', 'know', 'whats', 'gon', 'na', 'h...","['fuck', 'wrong', 'forgotten']","{'neg': 0.134, 'neu': 0.806, 'pos': 0.06, 'com...",-1,NaN,-0.5719,NaN
492,55,403,I'm loving these TD celebrations. Glad they're...,"['loving', 'Glad']","['Im', 'these', 'TD', 'celebrations', 'theyre'...",[],"{'neg': 0.0, 'neu': 0.465, 'pos': 0.535, 'comp...",1,NaN,0.7845,NaN
493,55,50,Love this hate Patriot fans.,['Love'],"['this', 'Patriot', 'fans']",['hate'],"{'neg': 0.339, 'neu': 0.275, 'pos': 0.385, 'co...",1,NaN,0.1280,NaN
494,55,215,STEPHEN GOSTKOWSKI IS STILL PERFECT THIS MONTH!,['PERFECT'],"['STEPHEN', 'GOSTKOWSKI', 'IS', 'STILL', 'THIS...",[],"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp...",1,NaN,0.5719,NaN


In [238]:
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/rohanjha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [263]:
token = word_tokenize(text)
token

['Bosa',
 'literally',
 'gets',
 'held',
 'next',
 'to',
 'that',
 'like',
 'come',
 'on']

In [264]:
from nltk.stem import PorterStemmer
pst = PorterStemmer()
stemmed_words = [pst.stem(x) for x in token]
stemmed_words

['bosa', 'liter', 'get', 'held', 'next', 'to', 'that', 'like', 'come', 'on']

In [265]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(x) for x in stemmed_words]
lemmatized_words

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rohanjha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['bosa', 'liter', 'get', 'held', 'next', 'to', 'that', 'like', 'come', 'on']

In [266]:
from nltk.corpus import stopwords
a = set(stopwords.words('english'))
no_stopwords = [x for x in lemmatized_words if x not in a]
no_stopwords

['bosa', 'liter', 'get', 'held', 'next', 'like', 'come']

In [267]:
nltk.download('averaged_perceptron_tagger')
for word in token:
    print(nltk.pos_tag([word]))

[('Bosa', 'NNP')]
[('literally', 'RB')]
[('gets', 'VBZ')]
[('held', 'NN')]
[('next', 'JJ')]
[('to', 'TO')]
[('that', 'IN')]
[('like', 'IN')]
[('come', 'VB')]
[('on', 'IN')]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rohanjha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [268]:
from nltk.probability import FreqDist
fdist = FreqDist([pst.stem(x) for x in no_stopwords])
fdist

FreqDist({'bosa': 1, 'liter': 1, 'get': 1, 'held': 1, 'next': 1, 'like': 1, 'come': 1})